<a href="https://colab.research.google.com/github/dimaswisodewo/Classification-using-ANN/blob/main/artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

## Importing the libraries

In [117]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [118]:
tf.__version__

'2.4.0'

# Part 1 - Data Preprocessing

## Importing the dataset

In [119]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
!ls "/content/drive/MyDrive/Colab Notebooks/Deep Learning/Artificial Neural Network (ANN)/"

artificial_neural_network.ipynb  Churn_Modelling.csv


In [121]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Artificial Neural Network (ANN)/Churn_Modelling.csv')

In [122]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [123]:
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [124]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [125]:
print(y)

[1 0 1 ... 1 1 0]


## Encoding Categorical Data

### Label Encoding the "Gender" column

In [126]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [127]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One Hot Encoding the "Geography" column

In [128]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = ct.fit_transform(X)

In [129]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [130]:
print(X[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


## Splitting the dataset into the Training Set and Testing Set

In [131]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Feature Scaling

In [132]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [133]:
print(X_train[:10])

[[-1.01460667 -0.5698444   1.74309049  0.16958176 -1.09168714 -0.46460796
   0.00666099 -1.21571749  0.8095029   0.64259497 -1.03227043  1.10643166]
 [-1.01460667  1.75486502 -0.57369368 -2.30455945  0.91601335  0.30102557
  -1.37744033 -0.00631193 -0.92159124  0.64259497  0.9687384  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 -1.19119591 -1.09168714 -0.94312892
  -1.031415    0.57993469 -0.92159124  0.64259497 -1.03227043  1.48533467]
 [-1.01460667 -0.5698444   1.74309049  0.03556578  0.91601335  0.10961719
   0.00666099  0.47312769 -0.92159124  0.64259497 -1.03227043  1.27652776]
 [-1.01460667 -0.5698444   1.74309049  2.05611444 -1.09168714  1.73658844
   1.04473698  0.8101927   0.8095029   0.64259497  0.9687384   0.55837842]
 [-1.01460667  1.75486502 -0.57369368  1.29325423 -1.09168714 -0.17749539
  -1.031415    0.44253504  0.8095029   0.64259497 -1.03227043  1.63252134]
 [ 0.98560362 -0.5698444  -0.57369368  1.6128308   0.91601335  0.77954653
  -1.37744033  0.30432823 -0.921

# Part 2 - Building the ANN

## Initializing the ANN

In [134]:
ann = tf.keras.models.Sequential()

## Adding the input layer and the first hidden layer

In [135]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding the second hidden layer

In [136]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding the output layer

In [137]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

## Compiling the ANN

In [138]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training the ANN on the Training Set

In [139]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)
# batch_size = 32 yang umum digunakan
# epochs tidak boleh terlalu sedikit

Epoch 1/100
250/250 [==============================] - 1s 965us/step - loss: 0.5802 - accuracy: 0.7401
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4712 - accuracy: 0.7912
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4481 - accuracy: 0.7968
Epoch 4/100
250/250 [==============================] - 0s 983us/step - loss: 0.4383 - accuracy: 0.7950
Epoch 5/100
250/250 [==============================] - 0s 968us/step - loss: 0.4302 - accuracy: 0.7985
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4189 - accuracy: 0.8028
Epoch 7/100
250/250 [==============================] - 0s 998us/step - loss: 0.4021 - accuracy: 0.8152
Epoch 8/100
250/250 [==============================] - 0s 937us/step - loss: 0.3905 - accuracy: 0.8197
Epoch 9/100
250/250 [==============================] - 0s 967us/step - loss: 0.3803 - accuracy: 0.8300
Epoch 10/100
250/250 [==============================] - 0s 937us/step - loss: 0

# Part 4 - Making the predictions and evaluating the model

## Predicting result of a single observation

In [140]:
data = np.array([[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000]])
data[:,2] = le.transform(data[:,2]) 
data = ct.transform(data)
data = sc.transform(data)
print(data)

[[ 0.98560362 -0.5698444  -0.57369368 -0.52111599  0.91601335  0.10961719
  -0.68538967 -0.2569057   0.8095029   0.64259497  0.9687384  -0.87203322]]


In [143]:
print(ann.predict(data))

[[0.03830147]]


In [144]:
print(ann.predict(data) > 0.5)

[[False]]


## Predicting the Test Set results

In [158]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusion Matrix

In [159]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))

[[1514   81]
 [ 195  210]]
Accuracy:  0.862
